In [1]:
import polars as pl
import tarfile
import os
import datetime as dt
from pathlib import Path
from tqdm import tqdm

from src.preprocessing import preprocess_all_tickers, select_complete_tickers, count_tickers_with_expected_rows

# Autorealod extension for Jupyter Notebooks
%load_ext autoreload
%autoreload 2

In [2]:
preprocess_all_tickers()

2025-12-01 13:35:19,233 | INFO | Starting preprocessing of all tickers...
2025-12-01 13:35:19,322 | INFO | Scanning RAW_FOLDER for tickers...
2025-12-01 13:35:19,358 | INFO | Found 102 tickers.
2025-12-01 13:35:19,362 | INFO | 9 tickers need preprocessing.


Preprocessing tickers:   0%|          | 0/9 [00:00<?, ?it/s]

2025-12-01 13:35:19,378 | INFO | === Preprocessing ticker BRK.B.OQ ===
2025-12-01 13:35:19,378 | INFO | [BRK.B.OQ] Extracting archives...
2025-12-01 13:35:20,309 | INFO | Loading parquet files in data\extracted\SP100\bbo\BRK.B.OQ...
2025-12-01 13:35:20,309 | INFO | Loaded 0 parquet files.
2025-12-01 13:35:20,309 | ERROR | No parquet files found for ticker BRK.B.OQ
2025-12-01 13:35:20,309 | ERROR | [BRK.B.OQ] Error during preprocessing: No parquet files found for ticker BRK.B.OQ
2025-12-01 13:35:20,321 | INFO | === Preprocessing ticker XOM.OQ ===
2025-12-01 13:35:20,321 | INFO | [XOM.OQ] Extracting archives...
2025-12-01 13:35:20,470 | INFO | Loading parquet files in data\extracted\SP100\bbo\XOM.OQ...
2025-12-01 13:35:20,470 | INFO | Loaded 0 parquet files.
2025-12-01 13:35:20,470 | ERROR | No parquet files found for ticker XOM.OQ
2025-12-01 13:35:20,470 | ERROR | [XOM.OQ] Error during preprocessing: No parquet files found for ticker XOM.OQ
2025-12-01 13:35:20,470 | INFO | === Preproces

In [3]:
select_complete_tickers(exception_dates=[dt.datetime(2015, 11, 27).date()], delete_exception_days=True)

2025-12-01 13:35:24,636 | INFO | Starting selection of complete tickers...
2025-12-01 13:35:25,568 | WARNING | [AGN.N.parquet] Missing 62 trading days: [datetime.date(2015, 3, 17), datetime.date(2015, 3, 18), datetime.date(2015, 3, 19), datetime.date(2015, 3, 20), datetime.date(2015, 3, 23), datetime.date(2015, 3, 24), datetime.date(2015, 3, 25), datetime.date(2015, 3, 26), datetime.date(2015, 3, 27), datetime.date(2015, 3, 30), datetime.date(2015, 3, 31), datetime.date(2015, 4, 1), datetime.date(2015, 4, 2), datetime.date(2015, 4, 6), datetime.date(2015, 4, 7), datetime.date(2015, 4, 8), datetime.date(2015, 4, 9), datetime.date(2015, 4, 10), datetime.date(2015, 4, 13), datetime.date(2015, 4, 14), datetime.date(2015, 4, 15), datetime.date(2015, 4, 16), datetime.date(2015, 4, 17), datetime.date(2015, 4, 20), datetime.date(2015, 4, 21), datetime.date(2015, 4, 22), datetime.date(2015, 4, 23), datetime.date(2015, 4, 24), datetime.date(2015, 4, 27), datetime.date(2015, 4, 28), datetime.date

In [4]:
count_tickers_with_expected_rows()

2025-12-01 13:35:40,693 | INFO | Counting rows for each selected ticker...
2025-12-01 13:35:42,150 | INFO | Number of tickers with the expected number of rows: 90 out of 90
